##  Data Preprocessing



In [16]:
# Combine the two csv(two labels) into one csv file they have the same column names.
import pandas as pd



table_csv = 'inertial_data_table_1730446449.2779398.csv'
hand_csv = 'inertial_data_hand_1730446565.1342528.csv'

#  due the 0 value in the first session, delete the session_id=0 for both csv files
table_df = pd.read_csv(table_csv)
table_df = table_df[table_df['session_id'] != 0]
table_df.to_csv('inertial_data_table_processed.csv', index=False)

hand_df = pd.read_csv(hand_csv)
hand_df = hand_df[hand_df['session_id'] != 0]
hand_df.to_csv('inertial_data_hand_processed.csv', index=False)

#  combine the two csv files into one csv file
table_df = pd.read_csv('inertial_data_table_processed.csv')
hand_df = pd.read_csv('inertial_data_hand_processed.csv')
combined_df = pd.concat([table_df, hand_df], ignore_index=True)
combined_df.to_csv('inertial_data_combined_processed.csv', index=False)



In [17]:
# csv file ready
data_csv = 'inertial_data_combined_processed.csv'




### 1D CNN Model


In [18]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [19]:
# create a 1D CNN model 
from sklearn.preprocessing import StandardScaler
from datetime import datetime

data_df = pd.read_csv(data_csv)

all_columns = data_df.columns.tolist() 
print(all_columns)



#define the features and label
features = all_columns[:-1]
label = all_columns[-1]

print(features)
print(label)
print(data_df.head())

x=data_df[features]
y=data_df[label]

print(x.shape)
print(y.shape)

# normalize the features using standard scaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Reshape the data for the CNN model
x_reshaped = x_scaled.reshape(x_scaled.shape[0], x_scaled.shape[1], 1)




['timestamp', 'gyroscope_pitch', 'gyroscope_yaw', 'gyroscope_roll', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z', 'session_id', 'label']
['timestamp', 'gyroscope_pitch', 'gyroscope_yaw', 'gyroscope_roll', 'accelerometer_x', 'accelerometer_y', 'accelerometer_z', 'session_id']
label
      timestamp  gyroscope_pitch  gyroscope_yaw  gyroscope_roll  \
0  1.730446e+09             -475           8031            1101   
1  1.730446e+09             -475           8031            1101   
2  1.730446e+09             -475           8031            1101   
3  1.730446e+09             -521           8065            1113   
4  1.730446e+09             -521           8065            1113   

   accelerometer_x  accelerometer_y  accelerometer_z  session_id  label  
0                1                2                7           1  table  
1                1                2                7           1  table  
2                1                2                7           1  table  
3       

In [21]:
print(x_reshaped.shape)

print(x_reshaped.base)

(188863, 8, 1)
[[-1.66493383 -1.66491761 -1.66490161 ...  1.66537978  1.66539578
   1.66541219]
 [ 0.06504365  0.06504365  0.06504365 ...  0.74469173  0.74469173
   0.74469173]
 [ 0.79122393  0.79122393  0.79122393 ... -0.8102265  -0.8102265
  -0.8102265 ]
 ...
 [ 0.01607026  0.01607026  0.01607026 ...  0.7175835   0.7175835
   0.7175835 ]
 [-0.01348936 -0.01348936 -0.01348936 ... -0.47154587 -0.47154587
  -0.47154587]
 [-1.72326329 -1.72326329 -1.72326329 ...  1.7234668   1.7234668
   1.7234668 ]]
